In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.applications import ResNet50
import os
from sklearn.metrics import roc_curve, roc_auc_score
import cv2
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
import tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,BatchNormalization,GlobalAveragePooling2D,Dropout,MaxPooling2D,Attention,Conv2D,concatenate
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications import Xception,VGG16,VGG19,MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Layer, MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, LeakyReLU
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

In [2]:
train_path="/kaggle/input/deepfake-and-real-images/Dataset/Train"
test_path="/kaggle/input/deepfake-and-real-images/Dataset/Test"
val_path="/kaggle/input/deepfake-and-real-images/Dataset/Validation"

In [3]:
import os
import cv2 

def FrameCapture(path): 

    vidObj = cv2.VideoCapture(path) 

    # Used as counter variable 
    count = 0

    success = 1

    if not os.path.exists("frames"):
        os.makedirs("frames")

    while success: 

        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 

        # Check if the image is empty
        if not success:
            break

        # Saves the frames with frame-count 
        # Modifying to save frames with 'frame' prefix and count as multiples of 10
        frames_num = 20
        if count % frames_num == 0:
            cv2.imwrite("frames/frame%d.jpg" % (count), image) 

        count += 1


In [4]:
FrameCapture("/kaggle/input/test-video") 

In [ ]:
train_datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255.0)

# Data Generators
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),  # Adjusted target size
    batch_size=64,  # Increased batch size
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),  # Adjusted target size
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

val_set = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),  # Adjusted target size
    batch_size=32,
    class_mode='binary'
)


In [ ]:
from tensorflow.keras.layers import LSTM, TimeDistributed
# Define the LSTM configuration
from tensorflow.keras.layers import Reshape
lstm_units = 64  # Adjust as needed

# Define the CNN part of the model
base_model = Meso4(input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
cnn_output = Dense(16, activation='relu')(x)

# Reshape CNN output to fit LSTM input
reshaped_cnn_output = Reshape((1, 16))(cnn_output)  # Assuming output shape of Dense layer is (None, 16)

# Define the LSTM layer
lstm_layer = LSTM(units=lstm_units)

# Apply LSTM to the reshaped CNN output
lstm_output = lstm_layer(reshaped_cnn_output)

# Output layer
predictions = Dense(1, activation='sigmoid')(lstm_output)

# Combine the models
combined_model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
#combined_model.summary()

In [8]:
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)
combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
combined_model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
with tf.device('/GPU:1'):
    history = combined_model.fit(
        training_set,
        epochs=15,
        validation_data=val_set
    )


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Predict the classes using the test set
test_pred = combined_model.predict(test_set)
test_pred_classes = np.round(test_pred)

# Get the true classes of the test set
test_true_classes = test_set.classes

In [ ]:
# Plot confusion matrix as a heatmap
# Generate confusion matrix
cm = confusion_matrix(test_true_classes, test_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_set.class_indices.keys(),
            yticklabels=test_set.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
evaluation = combined_model.evaluate(test_set)

In [ ]:
print("Test Accuracy: {:.2f}%".format(evaluation[1] * 100))

In [ ]:
#Classification report
report = classification_report(test_true_classes, test_pred_classes, target_names=["Class 0", "Class 1"], labels=[0, 1], digits=4)

print("Classification Report:\n", report)

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(test_true_classes, test_pred)

# Calculate AUC
auc = roc_auc_score(test_true_classes, test_pred)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (AUC = {:.4f})'.format(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
# Compute precision and recall
precision, recall, thresholds = precision_recall_curve(test_true_classes, test_pred_classes)

# Calculate the area under the precision-recall curve (AUC-PR)
auc_pr = auc(recall, precision)

# Plot the precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall Curve (AUC = {auc_pr:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

In [ ]:
# Compute confusion matrix
tn, fp, fn, tp = confusion_matrix(test_true_classes, test_pred_classes).ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f'Sensitivity (True Positive Rate): {sensitivity:.4f}')
print(f'Specificity (True Negative Rate): {specificity:.4f}')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# Assuming you have obtained test_pred_classes and test_true_classes as described earlier

# Calculate F1 score for the test classes
f1 = f1_score(test_true_classes, test_pred_classes)
# Calculate accuracy to get error rate
accuracy = accuracy_score(test_true_classes, test_pred_classes)
error_rate = 1 - accuracy
# Calculate G-mean
g_mean = (sensitivity * specificity) ** 0.5

# Calculate F-measure
f_measure = 2 * ((sensitivity * specificity) / (sensitivity + specificity))

print(f'F1 Score: {f1:.4f}')
print(f'Error Rate: {error_rate:.4f}')
print(f'G-mean: {g_mean:.4f}')
print(f'F-measure: {f_measure:.4f}')

In [23]:
import requests
from PIL import Image
from io import BytesIO

# image_url = "https://assets.bwbx.io/images/users/iqjWHBFdfxIU/irCSqgfWvpSw/v1/-1x-1.jpg"
image_url = "https://cdn.britannica.com/86/182086-050-5FB81069/singer-Taylor-swift-2013.jpg"
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))


In [24]:
img = img.resize((224, 224))  # Adjust size according to your model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values


In [ ]:
prediction = combined_model.predict(img_array)
if prediction[0][0] >= 0.5:
    print("The image is predicted as a deepfake.")
else:
    print("The image is predicted as real.")


In [ ]:
# Save the model
model_save_path = "deepfake_detection_model.h5"
combined_model.save(model_save_path)
print("Model saved at:", model_save_path)
